In [5]:
import pandas as pd

df = pd.read_csv('data/creditcard.csv')

df.head()

print("Raw:", df["Class"].value_counts().to_dict())   # {0: 284315, 1: 492}
y = 1 - df["Class"].values
print("After flip:", {0: int((y==0).sum()), 1: int((y==1).sum())})  # {0: 492, 1: 284315}


Raw: {0.0: 4410, 1.0: 2}
After flip: {0: 2, 1: 4410}


In [ ]:
# -*- coding: utf-8 -*-
# File: creditcard_ps_vs_baseline.py

import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import (
    f1_score, precision_score, recall_score, accuracy_score,
    average_precision_score, roc_auc_score
)
from sklearn.ensemble import RandomForestClassifier
import networkx as nx

# =========================
# 1) Load creditcard data
# =========================
# Expecting: data/creditcard.csv with columns:
# Time, V1..V28, Amount, Class (1=fraud minority, 0=legit majority)
df = pd.read_csv(os.path.join("data", "creditcard.csv"))

# Sanity check: show raw distribution
# print("Raw Class counts:", df["Class"].value_counts().to_dict())

# Our ACO code assumes: 0 = minority, 1 = majority
# In creditcard, Class==1 is minority (fraud). Flip it:
# new_y = 0 if fraud, 1 if normal
y_raw = df["Class"].values
y = 1 - y_raw  # fraud(1)->0, normal(0)->1

# features = everything except Class
features = [c for c in df.columns if c != "Class"]
X = df[features].values.astype(np.float64)

# =========================
# 2) Train/Test split + scale (fit scaler on TRAIN only)
# =========================
X_train_raw, X_test_raw, y_train, y_test, idx_train, idx_test = train_test_split(
    X, y, np.arange(len(df)),
    test_size=0.30,
    stratify=y,
    random_state=42
)

scaler = StandardScaler().fit(X_train_raw)
X_train = scaler.transform(X_train_raw)
X_test  = scaler.transform(X_test_raw)


# =========================
# 3) ACO on kNN graph (TRAIN only), then infer PS for TEST
# =========================
def run_aco_with_knn(
    X_train, y_train, X_test,
    k=5, n_ants=50, n_iter=30, rho=0.1, rng_seed=42,
    alpha_edge=1.0, beta_phero=1.0, tau=1.0,
    max_steps=100, eps=1e-12
):
    """
    Compute Pheromone-Score (PS) via kNN graph + ACO on TRAIN only (no leakage),
    then estimate PS for TEST by neighbor-weighted averaging.

    Assumptions:
      - X_train / X_test are already standardized (e.g., StandardScaler fit on train).

    y convention:
      - 0 = minority, 1 = majority

    Returns:
      pheromone_train_scaled (np.ndarray): PS in [0,1] for TRAIN nodes (shape: [n_train]).
      pheromone_test_scaled  (np.ndarray): PS estimate in [0,1] for TEST nodes  (shape: [n_test]).
    """
    # kNN on TRAIN
    nbrs = NearestNeighbors(n_neighbors=k, metric='euclidean').fit(X_train)
    dist_train, ind_train = nbrs.kneighbors(X_train)

    G = nx.Graph()
    n_tr = len(X_train)
    for i in range(n_tr):
        G.add_node(i, label=int(y_train[i]), pheromone=0.0)

    # build weighted edges using returned distances (avoid recomputing norms)
    for i, neigh in enumerate(ind_train):
        for jj, j in enumerate(neigh):
            if i == j:
                continue
            d_ij = dist_train[i, jj]
            w = 1.0 / (1.0 + d_ij)
            if G.has_edge(i, j):
                G[i][j]['weight'] = max(G[i][j]['weight'], w)
            else:
                G.add_edge(i, j, weight=w)

    majority_nodes = [i for i in range(n_tr) if y_train[i] == 1]
    minority_nodes = set([i for i in range(n_tr) if y_train[i] == 0])

    rng = np.random.default_rng(rng_seed)

    # ACO loop
    for _ in range(n_iter):
        for _ in range(n_ants):
            current = rng.choice(majority_nodes)
            visited = [current]
            steps = 0

            while steps < max_steps:
                neighs = [n for n in G.neighbors(current) if n not in visited]
                if not neighs:
                    break

                # move probability ∝ (edge_weight^alpha_edge) * ((1+pheromone)^beta_phero)
                raw = []
                for n in neighs:
                    e = G[current][n]['weight'] ** alpha_edge
                    p = (1.0 + G.nodes[n]['pheromone']) ** beta_phero
                    raw.append(e * p)
                raw = np.array(raw, dtype=float)

                # temperature-scaled softmax for stability/controllability
                logits = raw / max(tau, eps)
                logits -= logits.max()
                probs = np.exp(logits)
                s = probs.sum()
                if s <= eps:
                    break
                probs /= s

                next_node = rng.choice(neighs, p=probs)
                visited.append(next_node)
                current = next_node
                steps += 1

                if current in minority_nodes:
                    # deposit pheromone along the path
                    for idx in visited:
                        G.nodes[idx]['pheromone'] += 1.0
                    break

        # evaporation
        for n in G.nodes:
            G.nodes[n]['pheromone'] *= (1.0 - rho)

    # normalize PS on TRAIN
    pheromone_train = np.array([G.nodes[i]['pheromone'] for i in range(n_tr)], dtype=float)
    scaler = MinMaxScaler().fit(pheromone_train.reshape(-1, 1))
    pheromone_train_scaled = scaler.transform(pheromone_train.reshape(-1, 1)).ravel()

    # estimate PS for TEST via inverse-distance weighted average of TRAIN neighbors
    dist_te, ind_te = nbrs.kneighbors(X_test, n_neighbors=k)
    w_te = 1.0 / (eps + dist_te)
    w_te = w_te / (w_te.sum(axis=1, keepdims=True) + eps)
    pheromone_test_scaled = (w_te * pheromone_train_scaled[ind_te]).sum(axis=1)

    return pheromone_train_scaled, pheromone_test_scaled


# --------- Optional speed control for graph build ----------
# For very large training sets, you can cap the data used for the kNN graph
# and still infer PS for the full Train/Test by kNN to that subset.
max_train_for_graph = 60000  # adjust based on your machine
if len(X_train) > max_train_for_graph:
    rng = np.random.default_rng(42)
    keep = rng.choice(len(X_train), size=max_train_for_graph, replace=False)
    X_train_graph = X_train[keep]
    y_train_graph = y_train[keep]

    # Run ACO on the subset
    ps_train_sub, ps_test_est = run_aco_with_knn(
        X_train_graph, y_train_graph, X_test,
        k=10, n_ants=75, n_iter=25, rho=0.1, rng_seed=42,
        alpha_edge=1.0, beta_phero=1.0, tau=1.0,
        max_steps=50
    )

    # Now estimate PS for ALL train points by kNN to the subset
    nbrs_sub = NearestNeighbors(n_neighbors=10, metric='euclidean').fit(X_train_graph)
    dist_tr_all, ind_tr_all = nbrs_sub.kneighbors(X_train, n_neighbors=10)
    w_tr_all = 1.0 / (1e-12 + dist_tr_all)
    w_tr_all = w_tr_all / (w_tr_all.sum(axis=1, keepdims=True) + 1e-12)
    ps_train_all = (w_tr_all * ps_train_sub[ind_tr_all]).sum(axis=1)

    pheromone_train_scaled = ps_train_all
    pheromone_test_scaled  = ps_test_est
else:
    pheromone_train_scaled, pheromone_test_scaled = run_aco_with_knn(
        X_train, y_train, X_test,
        k=10, n_ants=75, n_iter=25, rho=0.1, rng_seed=42,
        alpha_edge=1.0, beta_phero=1.0, tau=1.0,
        max_steps=50
    )


# =========================
# 4) Build PS-weighted resample for RF
# =========================
def build_ps_weighted_resample(X_tr, y_tr, ps_tr,
                               target_ratio=1.0,
                               alpha=0.1, beta=0.9,
                               rng_seed=42):
    """
    Create a resampled training set where minority (label=0) samples are
    oversampled with probability ∝ (alpha + beta*PS).

    target_ratio: desired minority/majority count in the resampled set (1.0 → balanced)

    Returns: X_res, y_res
    """
    rng = np.random.default_rng(rng_seed)
    idx_min = np.where(y_tr == 0)[0]
    idx_maj = np.where(y_tr == 1)[0]

    n_min = len(idx_min)
    n_maj = len(idx_maj)

    # how many minority samples we want
    n_min_target = int(target_ratio * n_maj)

    # sampling probabilities for minority based on PS
    ps_min = ps_tr[idx_min]
    probs = alpha + beta * ps_min
    probs = np.maximum(probs, 1e-12)
    probs = probs / probs.sum()

    # sample with replacement to reach target
    if n_min_target <= n_min:
        chosen_min = rng.choice(idx_min, size=n_min_target, replace=False, p=probs)
    else:
        chosen_min = rng.choice(idx_min, size=n_min_target, replace=True,  p=probs)

    # keep all majority (or you can also bootstrap majority if you prefer)
    chosen_maj = idx_maj

    idx_res = np.concatenate([chosen_min, chosen_maj])
    rng.shuffle(idx_res)

    return X_tr[idx_res], y_tr[idx_res]


X_tr_ps, y_tr_ps = build_ps_weighted_resample(
    X_train, y_train, pheromone_train_scaled,
    target_ratio=1.0,   # make minority ~= majority
    alpha=0.1, beta=0.9,
    rng_seed=42
)

# =========================
# 5) Train & evaluate
# =========================
def prob_of_class0(model, X):
    """Return P(y=0) robustly using model.classes_ ordering."""
    cls = list(model.classes_)
    idx0 = cls.index(0)
    return model.predict_proba(X)[:, idx0]

def evaluate(y_true, y_prob_class0, thresh=0.5):
    """
    y_prob_class0: predicted probability for class 0 (fraud, positive)
    Decision rule: ŷ = 0 if P(class=0) >= thresh else 1
    """
    y_pred = np.where(y_prob_class0 >= thresh, 0, 1)
    y_pos = (y_true == 0).astype(int)

    return {
        "F1":        f1_score(y_true, y_pred, pos_label=0),
        "Precision": precision_score(y_true, y_pred, pos_label=0, zero_division=0),
        "Recall":    recall_score(y_true, y_pred, pos_label=0),
        "Accuracy":  accuracy_score(y_true, y_pred),
        "AUC_PR":    average_precision_score(y_pos, y_prob_class0),
        "ROC_AUC":   roc_auc_score(y_pos, y_prob_class0),
    }

# Baseline RF (no class_weight, no PS)
rf_base = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)
rf_base.fit(X_train, y_train)
proba_base = prob_of_class0(rf_base, X_test)
metrics_base = evaluate(y_test, proba_base, thresh=0.5)

# PS-weighted resampled RF
rf_ps = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)
rf_ps.fit(X_tr_ps, y_tr_ps)
proba_ps = prob_of_class0(rf_ps, X_test)
metrics_ps = evaluate(y_test, proba_ps, thresh=0.5)

# =========================
# 6) Report
# =========================
def pretty(d):
    return " | ".join(f"{k}: {v:.4f}" for k, v in d.items())

print("\n=== Results (positive class = fraud → label 0) ===")
print("Baseline RF           :", pretty(metrics_base))
print("RF + PS-weighted SMPL :", pretty(metrics_ps))


In [ ]:
# -*- coding: utf-8 -*-
"""
Lightweight ACO+kNN PS on creditcard dataset with minimal compute.
- 0 = minority (fraud, positive), 1 = majority (normal)
- Strategies: baseline RF, RF + PS(sample_weight), RF + PS-guided resampling
"""

import os
import time
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from collections import defaultdict

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    f1_score, precision_score, recall_score, accuracy_score,
    average_precision_score, roc_auc_score
)
import networkx as nx

# =========================
# 0) Load data + flip labels
# =========================
df = pd.read_csv(os.path.join("data", "creditcard.csv"))
# creditcard: Class==1 (fraud), Class==0 (normal)
# Convention here: 0 = minority (fraud, positive), 1 = majority
y = 1 - df["Class"].values
X = df.drop(columns=["Class"]).values.astype(np.float64)

# =========================
# 1) Configuration (LIGHT MODE)
# =========================
N_FOLDS = 3                     # fewer than 5 for lighter runs
SEED = 42
np.random.seed(SEED)

# Subset of Train used to build ACO graph in each fold:
# - include all minority samples
# - cap majority samples at cap_maj (undersample)
cap_maj_for_graph_per_fold = 35000  # lower this if computation is heavy
k_values  = [5, 10, 20, 30]         # small grid
n_ants    = 100                     # fewer ants
n_iter    = 50                      # fewer iterations
rho       = 0.1
alpha_edge = 1.0
beta_phero = 1.0
tau        = 1.0
max_steps_scale = 20                # max_steps = max_steps_scale * k

# RF configuration
rf_n_estimators = 150
rf_random_state = 42

# PS-guided resampling
target_ratio = 1.0  # target minority/majority ratio (≈ balanced)
alpha = 0.1         # w_i ∝ alpha + beta * PS
beta  = 0.9

# =========================
# 2) Helper functions
# =========================
def evaluate_pos0(y_true, y_prob_class0, thresh=0.5):
    """Metrics with positive class = 0 (fraud). y_prob_class0 = P(y=0)."""
    y_pred = np.where(y_prob_class0 >= thresh, 0, 1)
    y_pos = (y_true == 0).astype(int)
    return {
        "F1":        f1_score(y_true, y_pred, pos_label=0),
        "Precision": precision_score(y_true, y_pred, pos_label=0, zero_division=0),
        "Recall":    recall_score(y_true, y_pred, pos_label=0),
        "Accuracy":  accuracy_score(y_true, y_pred),
        "AUC_PR":    average_precision_score(y_pos, y_prob_class0),
        "ROC_AUC":   roc_auc_score(y_pos, y_prob_class0),
    }

def pretty(d):
    return " | ".join(f"{k}: {v:.4f}" for k, v in d.items())

def ps_guided_resample(X_tr, y_tr, ps_tr, target_ratio=1.0, alpha=0.1, beta=0.9, seed=42):
    """Oversample minority (0) with probabilities ∝ alpha + beta*PS to reach target_ratio."""
    rng = np.random.default_rng(seed)
    idx_min = np.where(y_tr == 0)[0]
    idx_maj = np.where(y_tr == 1)[0]
    n_maj = len(idx_maj)
    n_min_target = int(target_ratio * n_maj)
    ps_min = ps_tr[idx_min]
    probs = alpha + beta * ps_min
    probs = np.maximum(probs, 1e-12)
    probs = probs / probs.sum()
    if n_min_target <= len(idx_min):
        chosen_min = rng.choice(idx_min, size=n_min_target, replace=False, p=probs)
    else:
        chosen_min = rng.choice(idx_min, size=n_min_target, replace=True, p=probs)
    idx_new = np.concatenate([idx_maj, chosen_min])
    rng.shuffle(idx_new)
    return X_tr[idx_new], y_tr[idx_new]

# =========================
# 3) ACO on kNN (LIGHT): precompute once per fold, reuse for different k via k_max
# =========================
def run_aco_with_knn_light(X_tr_graph, y_tr_graph, X_te,
                           k, n_ants, n_iter, rho,
                           alpha_edge, beta_phero, tau,
                           max_steps, eps=1e-12,
                           precomp=None):
    """
    Light ACO on a pre-capped graph subset.
    If precomp contains a fitted NearestNeighbors with k_max, reuses it and slices to k.
    """
    # Fit or reuse neighbors (k_max)
    if precomp is None or "nbrs" not in precomp or precomp["k_max"] < k:
        k_max = max(k, 40)  # build a single model with larger k to slice from
        nbrs = NearestNeighbors(n_neighbors=k_max, metric='euclidean').fit(X_tr_graph)
        dist_tr_all, ind_tr_all = nbrs.kneighbors(X_tr_graph)
        precomp = {"nbrs": nbrs, "dist_tr_all": dist_tr_all, "ind_tr_all": ind_tr_all, "k_max": k_max}
    else:
        k_max = precomp["k_max"]
        nbrs = precomp["nbrs"]
        dist_tr_all = precomp["dist_tr_all"]
        ind_tr_all = precomp["ind_tr_all"]

    # Slice first-k
    dist_train = dist_tr_all[:, :k]
    ind_train  = ind_tr_all[:, :k]

    # Build graph
    G = nx.Graph()
    n_tr = len(X_tr_graph)
    for i in range(n_tr):
        G.add_node(i, label=int(y_tr_graph[i]), pheromone=0.0)

    for i, neigh in enumerate(ind_train):
        for jj, j in enumerate(neigh):
            if i == j:
                continue
            d_ij = dist_train[i, jj]
            w = 1.0 / (1.0 + d_ij)
            if G.has_edge(i, j):
                G[i][j]['weight'] = max(G[i][j]['weight'], w)
            else:
                G.add_edge(i, j, weight=w)

    majority_nodes = [i for i in range(n_tr) if y_tr_graph[i] == 1]
    minority_nodes = set([i for i in range(n_tr) if y_tr_graph[i] == 0])

    rng = np.random.default_rng(123)
    for _ in range(n_iter):
        for _ in range(n_ants):
            current = rng.choice(majority_nodes)
            visited = [current]
            steps = 0
            while steps < max_steps:
                neighs = [n for n in G.neighbors(current) if n not in visited]
                if not neighs:
                    break
                raw = []
                for n in neighs:
                    e = G[current][n]['weight'] ** alpha_edge
                    p = (1.0 + G.nodes[n]['pheromone']) ** beta_phero
                    raw.append(e * p)
                raw = np.array(raw, dtype=float)
                logits = raw / max(tau, eps)
                logits -= logits.max()
                probs = np.exp(logits)
                s = probs.sum()
                if s <= eps:
                    break
                probs /= s
                next_node = rng.choice(neighs, p=probs)
                visited.append(next_node)
                current = next_node
                steps += 1
                if current in minority_nodes:
                    for idx in visited:
                        G.nodes[idx]['pheromone'] += 1.0
                    break
        for n in G.nodes:
            G.nodes[n]['pheromone'] *= (1.0 - rho)

    # Normalize PS on TRAIN(subset)
    pheromone_train = np.array([G.nodes[i]['pheromone'] for i in range(n_tr)], dtype=float)
    scaler = MinMaxScaler().fit(pheromone_train.reshape(-1, 1))
    phero_train_scaled = scaler.transform(pheromone_train.reshape(-1, 1)).ravel()

    # Infer PS for TEST w.r.t. graph subset
    dist_te_all, ind_te_all = precomp["nbrs"].kneighbors(X_te, n_neighbors=k)
    w_te = 1.0 / (1e-12 + dist_te_all)
    w_te = w_te / (w_te.sum(axis=1, keepdims=True) + 1e-12)
    phero_test_scaled = (w_te * phero_train_scaled[ind_te_all]).sum(axis=1)

    return phero_train_scaled, phero_test_scaled, precomp

# =========================
# 4) Cross-validation split + per-fold graph subset
# =========================
cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
folds = []
for f, (tr_idx, te_idx) in enumerate(cv.split(X, y)):
    X_tr_raw, X_te_raw = X[tr_idx], X[te_idx]
    y_tr, y_te = y[tr_idx], y[te_idx]
    scaler = StandardScaler().fit(X_tr_raw)
    X_tr = scaler.transform(X_tr_raw)
    X_te = scaler.transform(X_te_raw)

    # Build lighter graph subset: all minority + capped majority
    idx_min = np.where(y_tr == 0)[0]
    idx_maj = np.where(y_tr == 1)[0]

    if len(idx_maj) > cap_maj_for_graph_per_fold:
        rng = np.random.default_rng(SEED + f)
        idx_maj_cap = rng.choice(idx_maj, size=cap_maj_for_graph_per_fold, replace=False)
    else:
        idx_maj_cap = idx_maj

    keep_idx = np.concatenate([idx_min, idx_maj_cap])
    X_tr_graph = X_tr[keep_idx]
    y_tr_graph = y_tr[keep_idx]

    folds.append({
        "X_tr": X_tr, "y_tr": y_tr, "X_te": X_te, "y_te": y_te,
        "X_tr_graph": X_tr_graph, "y_tr_graph": y_tr_graph
    })

# =========================
# 5) Precompute PS for each fold, for a small set of k
# =========================
# We reuse a neighbor model with k_max and slice to any k <= k_max
k_max = max(k_values)
ps_cache = defaultdict(dict)  # ps_cache[f][k] = (ps_train_full, ps_test)

for f, pack in enumerate(folds):
    X_tr, y_tr = pack["X_tr"], pack["y_tr"]
    X_te, y_te = pack["X_te"], pack["y_te"]
    X_tr_g, y_tr_g = pack["X_tr_graph"], pack["y_tr_graph"]

    precomp = None
    for k in sorted(k_values):
        # Run light ACO on subset
        phero_tr_sub, phero_te, precomp = run_aco_with_knn_light(
            X_tr_g, y_tr_g, X_te,
            k=k, n_ants=n_ants, n_iter=n_iter, rho=rho,
            alpha_edge=alpha_edge, beta_phero=beta_phero, tau=tau,
            max_steps=max_steps_scale * k,
            precomp=precomp
        )
        # Map PS back to FULL train via kNN-to-subset (10-NN)
        nbrs_sub = NearestNeighbors(n_neighbors=min(10, len(X_tr_g)), metric='euclidean').fit(X_tr_g)
        dist_tr_all, ind_tr_all = nbrs_sub.kneighbors(X_tr, n_neighbors=min(10, len(X_tr_g)))
        w_tr_all = 1.0 / (1e-12 + dist_tr_all)
        w_tr_all = w_tr_all / (w_tr_all.sum(axis=1, keepdims=True) + 1e-12)
        phero_tr_full = (w_tr_all * phero_tr_sub[ind_tr_all]).sum(axis=1)

        ps_cache[f][k] = (phero_tr_full, phero_te)

# =========================
# 6) Train/Evaluate (3 strategies)
# =========================
def run_fold_strategies(fpack, k, rf_n_estimators):
    X_tr, y_tr = fpack["X_tr"], fpack["y_tr"]
    X_te, y_te = fpack["X_te"], fpack["y_te"]
    phero_tr, phero_te = ps_cache[fpack["id"]][k]

    # 6.1 Baseline RF (without PS)
    rf_base = RandomForestClassifier(
        n_estimators=rf_n_estimators,
        class_weight='balanced_subsample',
        random_state=rf_random_state,
        n_jobs=-1
    )
    rf_base.fit(X_tr, y_tr)
    proba0_base = rf_base.predict_proba(X_te)[:, list(rf_base.classes_).index(0)]
    met_base = evaluate_pos0(y_te, proba0_base)

    # 6.2 RF + PS as feature + sample_weight
    X_tr_w = np.column_stack([X_tr, phero_tr])
    X_te_w = np.column_stack([X_te, phero_te])
    weights = np.where(y_tr == 0, 1.0 + 2.0 * phero_tr, 1.0 + 0.5 * phero_tr)
    rf_w = RandomForestClassifier(
        n_estimators=rf_n_estimators,
        random_state=rf_random_state,
        n_jobs=-1
    )
    rf_w.fit(X_tr_w, y_tr, sample_weight=weights)
    proba0_w = rf_w.predict_proba(X_te_w)[:, list(rf_w.classes_).index(0)]
    met_w = evaluate_pos0(y_te, proba0_w)

    # 6.3 RF + PS-guided resampling
    X_tr_rs, y_tr_rs = ps_guided_resample(X_tr_w, y_tr, phero_tr, target_ratio=target_ratio, alpha=alpha, beta=beta, seed=SEED)
    rf_rs = RandomForestClassifier(
        n_estimators=rf_n_estimators,
        random_state=rf_random_state,
        n_jobs=-1
    )
    rf_rs.fit(X_tr_rs, y_tr_rs)
    proba0_rs = rf_rs.predict_proba(X_te_w)[:, list(rf_rs.classes_).index(0)]
    met_rs = evaluate_pos0(y_te, proba0_rs)

    return met_base, met_w, met_rs

# Attach index id to folds
for i in range(len(folds)):
    folds[i]["id"] = i

all_rows = []
t0 = time.perf_counter()
for k in k_values:
    base_list, w_list, rs_list = [], [], []
    for fpack in folds:
        m_base, m_w, m_rs = run_fold_strategies(fpack, k, rf_n_estimators)
        base_list.append(m_base)
        w_list.append(m_w)
        rs_list.append(m_rs)

    def avg(metrics_list):
        keys = list(metrics_list[0].keys())
        return {f"{k}_mean": float(np.mean([m[k] for m in metrics_list]))
                for k in keys} | {f"{k}_std": float(np.std([m[k] for m in metrics_list]))
                for k in keys}

    res_base = {"strategy": "rf_baseline", "k": k} | avg(base_list)
    res_w    = {"strategy": "rf_weight",   "k": k} | avg(w_list)
    res_rs   = {"strategy": "rf_resample", "k": k} | avg(rs_list)
    all_rows.extend([res_base, res_w, res_rs])

t1 = time.perf_counter()
results_df = pd.DataFrame(all_rows)

# =========================
# 7) Report results
# =========================
show_cols = ["strategy", "k",
             "F1_mean", "Precision_mean", "Recall_mean",
             "AUC_PR_mean", "ROC_AUC_mean", "Accuracy_mean"]

print(f"[Light run] time: {t1 - t0:.1f}s, folds={N_FOLDS}, k_values={k_values}, "
      f"cap_maj_for_graph_per_fold={cap_maj_for_graph_per_fold}")

print("\n=== Summary (means over folds) ===")
def pr(df):
    print(df[show_cols].to_string(index=False))

pr(results_df.sort_values(["strategy","k"]))

# Optional: save results
# results_df.to_csv("light_ps_results.csv", index=False)
